# Stock Analysis with yfinance

In [1]:
!pip install yfinance
!pip install bs4
!pip install plotly
!pip install requests
!pip install pandas

  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24095 sha256=526775ca43d17262cad9f1bd5f52279dfa109daaf6730612a4444c2b79680434
  Stored in directory: /Users/nevillejeffree/Library/Caches/pip/wheels/f9/e9/7e/8b13db3bf3aeb5049d759e10702736fb96753089ac950fddc0
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8368 sha256=02d278c45aac4f308f6f9ce94082ca091e1d294f34dad9a2557860c35ed94528
  Stored in directory: /Users/nevillejeffree/Library/Caches/pip/wheels/57/6d/a3/a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built yfinance multitasking


In [2]:
import pandas as pd
import requests
import yfinance as yf
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [12]:
starbucks = yf.Ticker('SBUX')
starbucks_info = starbucks.info
starbucks_data = starbucks.history(period = 'max')

In [20]:
starbucks_data.reset_index(inplace = True)
starbucks_data.head()

,index,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,0,1992-06-26,0.270958,0.287086,0.264507,0.277409,224358400,0.0,0.0
1,1,1992-06-29,0.280635,0.303215,0.274183,0.296763,58732800,0.0,0.0
2,2,1992-06-30,0.303215,0.306440,0.283860,0.287086,34777600,0.0,0.0
3,3,1992-07-01,0.290312,0.296763,0.280635,0.293537,18316800,0.0,0.0
4,4,1992-07-02,0.296763,0.296763,0.287086,0.293537,13996800,0.0,0.0


In [8]:
url = 'https://www.macrotrends.net/stocks/charts/SBUX/starbucks/revenue'

In [9]:
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')
tables = soup.find_all('table')

In [10]:
for index,table in enumerate(tables):
    if ("Starbucks Quarterly Revenue" in str(table)):
        table_index = index
print(table_index)

1


In [14]:
sbux_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text
        
        sbux_revenue = sbux_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

sbux_revenue

,Date,Revenue
0,2021-09-30,"$8,147"
1,2021-06-30,"$7,497"
2,2021-03-31,"$6,668"
3,2020-12-31,"$6,749"
4,2020-09-30,"$6,203"
...,...,...
62,2006-03-31,"$1,886"
63,2005-12-31,"$1,934"
64,2005-09-30,"$1,659"
65,2005-06-30,"$1,602"


In [15]:
sbux_revenue["Revenue"] = sbux_revenue['Revenue'].str.replace(',|\$',"")

<ipython-input-15-14840dd0959a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sbux_revenue["Revenue"] = sbux_revenue['Revenue'].str.replace(',|\$',"")


In [16]:
sbux_revenue.dropna(inplace=True)

sbux_revenue = sbux_revenue[sbux_revenue['Revenue'] != ""]

In [17]:
sbux_revenue

,Date,Revenue
0,2021-09-30,8147
1,2021-06-30,7497
2,2021-03-31,6668
3,2020-12-31,6749
4,2020-09-30,6203
...,...,...
62,2006-03-31,1886
63,2005-12-31,1934
64,2005-09-30,1659
65,2005-06-30,1602
